In [ ]:
import os
import time
import requests
import pandas as pd
import numpy as np
from pwe import charts,analysis,quan,cmc,pwetools
from datetime import datetime, date, timedelta, timezone
from alpha_vantage.cryptocurrencies import CryptoCurrencies

In [ ]:
# https://strftime.org/
td = date.today()
today = td.strftime("%d-%m-%Y")
today_dmy = str(today)

utc_now = datetime.utcnow()
utc_now.isoformat()
today_utc = utc_now.strftime("%d-%m-%Y")

end_date = today_dmy
end = datetime.strptime(td.strftime('%Y-%m-%d'),'%Y-%m-%d')

# utc_end = pd.to_datetime(utc_end)
utc_end = pd.Timestamp.utcnow()

interval = timedelta(1)
yesterday = td - interval
yest = yesterday.strftime("%d-%m-%Y")
yesterday_dmy = str(yest)
print("Yesterday:",yesterday_dmy)
print("")

# Input start date:
#########################
start_date = "01-01-2017"
#########################

start = datetime.strptime("2017-01-01",'%Y-%m-%d')
utc_start = f"{start_date} 16:00:00.000 UTC"
#utc_start = datetime.strptime(utc_start,'%d-%m-%Y %H:%M:%S.%f %Z')
utc_start = pd.to_datetime(utc_start)
#utc_start = start.isoformat()

chart_start = utc_now - timedelta(days=365)
auto_start = chart_start.strftime("%d-%m-%Y")

print ("autostart:",auto_start)
print("")

ticker = "BTC"

print("Today's date:", today_dmy)
print (utc_now, "UTC")
print("")
print("Start date:", start_date)
print("Start time:",utc_start, "UTC")
print("")
print("End date:", end_date)
print (utc_end, "UTC")
print("")
print("Security:", ticker)

In [ ]:
av_api = 'ALPHAVANTAGE_API_KEY'
security = 'Bitcoin'
symbol = 'BTC'
market = 'USD'
interval = '60min'
chart_interval = 'Hourly'
period = 'Hour'
outputsize = 'full'
# outputsize='compact'

In [ ]:
tkr = symbol.replace('/', '_')

file = 'csv_files/BTC_USD_60min_full_2021-07-08_00,00,00-2021-08-18_19,00,00_asof_18-08-2021_23,21.csv'
file_path = os.path.abspath(file)

print (file_path)

In [ ]:
if os.path.exists(file_path) == True:
    
    print ("Reading recent file from CSV...")
    print (file_path)
    df = pd.read_csv(file_path,low_memory=False, index_col=['DateTime'], parse_dates=['DateTime','Date'],infer_datetime_format=True)
    df.name = symbol
    df
    
else:
    print ("No CSV found. Downloading data from API")
    cc = CryptoCurrencies(key=av_api,output_format='pandas') # retries=5
    # Get json object with the intraday data and another with  the call's metadata
    data, meta_data = cc.get_digital_currency_intraday(symbol, market, interval, outputsize)
    df = data.copy()
    pwetools.del_col_num(df)
    pwetools.sort_index(df)
    df.index.names = ['DateTime']
    df.name = symbol
    pwetools.df_to_csv(df,symbol,market,interval,outputsize)
    df
    

In [ ]:
df.info()

In [ ]:
start_date = "01-01-2017"
end_date = date.today().strftime("%d-%m-%Y")

annotations = dict({'2021-01-29': 'CME Exp.', '2021-02-26': 'CME Exp.', 
                    '2021-03-26': 'CME Exp.', '2021-04-30': 'CME Exp.', 
                    '2021-05-28': 'CME Exp.', '2021-06-25': 'CME Exp.', 
                    '2021-07-30': 'CME Exp.'})

auto_start = '2021-07-15'
auto_end = '2021-08-15'
asPlot = False
trading_periods = 365
market_hours = 24
interval = 'hourly'
p = 'Hr.'

In [ ]:
charts.quant_chart_int(df,start_date,end_date,ticker=symbol,
                       theme='white',auto_start=auto_start,auto_end=auto_end,
                       asPlot=asPlot,showlegend=True,boll_std=2,boll_periods=20,
                       showboll=True,showrsi=True,rsi_periods=14,showama=True,
                       ama_periods=9,showvol=True,show_price_range=True,
                       textangle=60,annots=None)

In [ ]:
time.sleep(1)

charts.pwe_line_chart(df,columns=['Close'],start_date=start_date,end_date=end_date,kind='scatter',
                      title=f'{security} {chart_interval} Close',
                  ticker=symbol,yTitle=f'{ticker}/{market}',asPlot=asPlot,showlegend=False,
                  theme='white',auto_start=auto_start,auto_end=auto_end,
                  connectgaps=False,annots=None)

In [ ]:
## Calculate Returns:
returns.get_returns(df, price='Close');

# A 30 day vol window for BTC is 720 hours, as this is hourly data.
# In this case, the window represets hours:
vol_window = 30
interval = 'hourly'

## Calculate 30 Day Volatility:
vol.get_vol(df, window=vol_window, column='Price_Returns',trading_periods=365,interval=interval);

## Calculate 30 Day YangZhang Volatility Estimator (Requires OHLC data):
vol.YangZhang_estimator(df, window=vol_window,trading_periods=365, clean=True,interval=interval);

In [ ]:
## Returns Profile:
stats = returns.return_stats(df,returns='Price_Returns',price='Close',
                                          trading_periods=365,interval=interval,market_hours=24,vol_window=vol_window);

In [ ]:
## PLot the Returns & Volatility Distributions:

dist = charts.pwe_return_dist_chart(df,start_date,end_date,tseries='Price_Returns',kind='scatter',
                                    title='Daily Bitcoin Returns on Bitfinex',ticker=symbol,yTitle=f'{symbol} (%)',
                                    asPlot=asPlot,showlegend=False,theme='white',auto_start=auto_start,
                                    auto_end=end_date,connectgaps=False,tickformat='.0%',decimals=2)

In [ ]:
dist_bar = charts.pwe_return_bar_chart(df,start_date,end_date,tseries='Price_Returns',kind='bar',
                                             title='Bitcoin Daily Returns',
                                             ticker=symbol,yTitle=f'{symbol} (%)',xTitle=None,asPlot=asPlot,theme='white',
                                             showlegend=False,auto_start=auto_start,auto_end=end_date,tickformat='.0%',
                                             decimals=2,orientation='v',textangle=0)

In [ ]:
yz30_dist = charts.pwe_return_dist_chart(df,start_date,end_date,
                                             tseries=f'YangZhang{vol_window}_{p}_Ann',kind='scatter',
                                    title='Spot Bitcoin to US Dollars: Annualized Yang-Zhang 30 Hour Volatility',
                                             ticker=f'{symbol} YZ Vol.',yTitle='BTC/USD YZ Vol.',
                                             asPlot=asPlot,showlegend=False,theme='white',
                                             auto_start=auto_start,auto_end=auto_end,
                                             connectgaps=False,tickformat='.0%',decimals=2)

In [ ]:
vol30_dist = charts.pwe_return_dist_chart(df,start_date,end_date,
                                              tseries=f'Ann_Vol_{vol_window}_{p}',kind='scatter',
                                    title='Spot Bitcoin to US Dollars: Annualized 30 Day Volatility',
                                             ticker=f'{symbol} Vol.',yTitle=f'{symbol} Vol.',asPlot=asPlot,
                                             showlegend=False,theme='white',auto_start=auto_start,auto_end=end_date,
                                             connectgaps=False,tickformat='.0%',decimals=2)



In [ ]:
## Returns by Subseries:
### 2021
df_2021 = returns.get_sub_series(df,start_date='2021-1-1',end_date='2021-12-31')
df_2021.name = 2021
df_2021

In [ ]:
returns.get_returns(df_2021, price='Close');
vol.YangZhang_estimator(df_2021, window=vol_window, trading_periods=365, clean=True,interval=interval);
vol.get_vol(df_2021, window=vol_window, column='Price_Returns',trading_periods=365,interval=interval);
df_2021

In [ ]:
stats_2021 = returns.return_stats(df_2021,returns='Price_Returns',price='Close',trading_periods=365,interval='hourly',market_hours=24);
